In [1]:
import os
import pandas as pd
from data_loader import get_patient_loader
import torch
from model import get_remote_models_and_path
from process_csv import merge_gene_data_and_coords

In [2]:
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"

data_dir_test = "../Test_Data/"
data_dir_train = "../Training_Data/"
patients_test  = [os.path.basename(f) for f in os.scandir(data_dir_test) if f.is_dir()]
patients_train = [os.path.basename(f) for f in os.scandir(data_dir_train) if f.is_dir()]
print(patients_test)
print(patients_train)

results_dir = "../results/"
gene = "RUBCNL"

In [3]:
# list of (model, path to .pt file)
models = get_remote_models_and_path(device)


In [4]:
for model in models:
    print(model[0].__class__.__name__)

In [5]:

for patient in patients_test:
    if os.path.exists(results_dir + "/" + gene + "_results_new.csv"):
        continue
    merge_gene_data_and_coords(data_dir_test,patient, results_dir, gene + "_results_new.csv", gene)
    
for patient in patients_train:
    if os.path.exists(results_dir + "/" + gene + "_results_new.csv"):
        continue
    merge_gene_data_and_coords(data_dir_train,patient, results_dir, gene + "_results_new.csv", gene)

In [5]:
def generate_results2(model, model_path, device, data_dir, results_dir, patient=None, gene="RUBCNL", results_filename=None):#
    if results_filename is None:
        results_filename = gene + "_results_new.csv"
    model.eval().to(device)
    loader = get_patient_loader(data_dir, patient, gene)
    filename = results_dir + "/" + patient + "/" + results_filename
    if os.path.exists(filename):
        file_df = pd.read_csv(filename)
        if model_path in file_df:
            print(model_path + " already calculated")
            return
    else:
        print(filename + " does not exist")
        return

    i = 0
    j = len(loader)
    with torch.no_grad():
        for images, _, name in loader:
            if i % 1000 == 0:
                print(i, "/", j)
            i += 1
            images = images.unsqueeze(0).to(device)
            images = images.float()

            output = model(images)
            output = output.squeeze().unsqueeze(0)
            file_df.loc[file_df['tile'] == os.path.basename(name),[model_path]] = output.item()

    file_df.to_csv(filename, index=False)

for model in models:
    for patient in patients_train:
        print(model[1],patient)
        generate_results2(model[0], model[1], device, data_dir_train, results_dir, patient)
        
for model in models:
    for patient in patients_test:
        if patient == "p021":
            continue
        print(model[1],patient)
        generate_results2(model[0], model[1], device, data_dir_test, results_dir, patient)